In [17]:
from flask import Flask, request, send_file
from PIL import Image
import io
import numpy as np
import cv2
app = Flask(__name__)

def process_image(image):
    # Add your image processing functions here
    # For example, you could convert the image to grayscale and invert the colors
    image = Image.fromarray(image).convert('L')
    image = np.array(image)
    image = 255 - image
    return image

@app.route('/process-image1', methods=['POST'])
def process_image_route():
    image_file = request.files.get('image')
    image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)
    # Process the image
    # detect objects in the image using DIP techniques and concepts
    ret, jpeg = cv2.imencode('.jpg', image)
    encoded_image = jpeg.tobytes()
    return send_file(io.BytesIO(encoded_image), mimetype='image/jpeg')
    # Return the encoded image as a response
    # return Response(encoded_image, mimetype='image/jpeg')
if __name__ == '__main__':
    #run app in debug mode on port 5000
    if(app.run( port=5000)):
        print("running")
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2023 01:45:36] "POST /process-image1 HTTP/1.1" 200 -
